<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_Pytorch_WithDeeplizard/blob/master/20_Callable_Neural_Networks_Linear_Layers_In_Depth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch Callable Neural Networks - Deep Learning In Python
在这一章中，我们将学习如何调用PyTorch神经网络模块，这意味着什么，以及它如何告知我们如何调用网络和层正向传播方法。

## 1.线性层如何工作
在上一节中我们了解了线性层如何使用矩阵乘法将其内部特征转换为外部特征。

* 当输入特征被线性层接收时，它们以平坦的一维张量的形式被接收，

* 然后乘以权重矩阵。 该矩阵乘法产生输出特征。

让我们在代码中看这个例子。



In [ ]:
import torch
import torch.nn as nn

In [ ]:
in_features = torch.tensor([1,2,3,4], dtype=torch.float32)

weight_matrix = torch.tensor([
    [1,2,3,4],
    [2,3,4,5],
    [3,4,5,6]
], dtype=torch.float32)

print(in_features.shape)
print(weight_matrix.shape)

torch.Size([4])
torch.Size([3, 4])


In [ ]:
weight_matrix.matmul(in_features)

tensor([30., 40., 50.])

在这里，我们创建了一个称为in_features的一维张量。 我们还创建了一个权重矩阵，它当然是二维张量。 然后，我们使用matmul（）函数来执行产生一维张量的矩阵乘法运算。

通常，权重矩阵定义一个线性函数，该函数将具有四个元素的一维张量映射到具有三个元素的一维张量。 我们可以将此函数视为从4维欧几里德空间到3维欧几里德空间的映射。

这也是线性层如何工作的方式。 他们使用权重矩阵将in_feature空间映射到out_feature空间。

### 使用PyTorch线性层进行变换
让我们看看如何创建一个PyTorch线性图层来执行相同的操作。

In [ ]:
fc = nn.Linear(in_features=4,out_features=3,bias=False);

在这里我们定义了一个线性图层，该图层接受4个输入要素并将其转换为3个输出要素，因此我们从4维空间转换为3维空间。 我们知道，权重矩阵用于执行此操作，但是此示例中的权重矩阵在哪里？

我们将权重矩阵存在于PyTorch LinearLayer类中，并由PyTorch创建。 PyTorch LinearLayer类使用传递给构造函数的数字4和3创建3 x 4权重矩阵。 让我们通过查看PyTorch源代码来验证这一点。

In [ ]:
# torch/nn/modules/linear.py (version 1.0.1)

def __init__(self, in_features, out_features, bias=True):
    super(Linear, self).__init__()
    self.in_features = in_features
    self.out_features = out_features
    self.weight = Parameter(torch.Tensor(out_features, in_features))
    if bias:
        self.bias = Parameter(torch.Tensor(out_features))
    else:
        self.register_parameter('bias', None)
    self.reset_parameters()

如我们所见，当我们将3 x 4矩阵与4 x 1矩阵相乘时，结果是3 x 1矩阵。 这就是为什么PyTorch以此方式构建权重矩阵的原因。 这些是用于矩阵乘法的线性代数规则。

让我们看看如何通过传递in_features张量来调用层。

In [ ]:
fc(in_features)

tensor([ 1.0890,  1.3523, -0.3886], grad_fn=<SqueezeBackward3>)

我们可以这样调用对象实例，因为PyTorch神经网络模块是可调用的Python对象。 我们将在一分钟内更仔细地研究这个重要的细节，但首先，请检查此输出。 实际上，我们确实获得了包含三个元素的一维张量。 但是，产生了不同的值。

这是因为PyTorch会创建一个权重矩阵，并使用随机值对其进行初始化。 这意味着两个示例的线性函数是不同的，因此我们使用不同的函数来生成这些输出。

请记住，权重矩阵内的值定义了线性函数。 这说明了在训练过程中，网络的映射如何随着权重的更新而变化。

让我们显式设置线性层的权重矩阵，使其与我们在其他示例中使用的权重矩阵相同。

In [ ]:
fc.weight

Parameter containing:
tensor([[-0.0453, -0.4760,  0.1914,  0.3780],
        [ 0.4686,  0.0408, -0.0371,  0.2283],
        [ 0.0007, -0.2560,  0.3350, -0.2206]], requires_grad=True)

In [ ]:
fc.weight.shape

torch.Size([3, 4])

In [ ]:
nn.Parameter(weight_matrix)

Parameter containing:
tensor([[1., 2., 3., 4.],
        [2., 3., 4., 5.],
        [3., 4., 5., 6.]], requires_grad=True)

In [ ]:
fc.weight = nn.Parameter(weight_matrix)

PyTorch模块权重需要作为参数。这就是为什么我们将权重矩阵张量包装在参数类实例中。现在让我们看看这个层如何使用新的权重矩阵变换输入。我们希望看到与前面例子相同的结果。

In [ ]:
fc(in_features)

tensor([30., 40., 50.], grad_fn=<SqueezeBackward3>)

## Callable Layers And Neural Networks

之前我们已经指出，调用layer对象实例就像调用函数一样有点奇怪。

In [ ]:
fc(in_features)

tensor([30., 40., 50.], grad_fn=<SqueezeBackward3>)

使得这成为可能的是PyTorch模块类实现了另一个特殊的Python函数**__call __（）**。 如果一个类实现了__call __（）方法，则在调用对象实例时将调用特殊的调用方法。

这个事实是一个重要的PyTorch概念，因为__call __（）方法与我们的图层和网络的forward（）方法交互的方式。

代替直接调用forward（）方法，我们调用对象实例。 调用对象实例后，在后台调用__call__（）方法，然后__call__（）依次调用forward（）方法。 这适用于所有PyTorch神经网络模块，即网络和层。

让我们在PyTorch源代码中看到这一点。

In [ ]:
# torch/nn/modules/module.py (version 1.0.1)
def __call__(self, *input, **kwargs):
    for hook in self._forward_pre_hooks.values():
        hook(self, input)
    if torch._C._get_tracing_state():
        result = self._slow_forward(*input, **kwargs)
    else:
        result = self.forward(*input, **kwargs)
    for hook in self._forward_hooks.values():
        hook_result = hook(self, input, result)
        if hook_result is not None:
            raise RuntimeError(
                "forward hooks should never return any values, but '{}'"
                "didn't return None".format(hook))
    if len(self._backward_hooks) > 0:
        var = result
        while not isinstance(var, torch.Tensor):
            if isinstance(var, dict):
                var = next((v for v in var.values() if isinstance(v, torch.Tensor)))
            else:
                var = var[0]
        grad_fn = var.grad_fn
        if grad_fn is not None:
            for hook in self._backward_hooks.values():
                wrapper = functools.partial(hook, self)
                functools.update_wrapper(wrapper, hook)
                grad_fn.register_hook(wrapper)
    return result

PyTorch在__call __（）方法中运行的额外代码是为什么我们从不直接调用forward（）方法的原因。 如果这样做，将不会执行其他PyTorch代码。 结果，每当我们要调用forward（）方法时，我们都将调用对象实例。 这适用于层和网络，因为它们都是PyTorch神经网络模块。

现在，我们准备实现网络的forward（）方法。